In [24]:
import pandas as pd
import numpy as np

In [54]:
data = pd.read_excel("Online Retail-unclean.xlsx")

In [51]:
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541904 non-null  float64       
 6   CustomerID   406823 non-null  float64       
 7   Country      541901 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [28]:
persentase_missing = data.isnull().mean() * 100
persentase_missing = persentase_missing.sort_values(ascending=False)

print("Persentase Missing Values : ")
print(persentase_missing)

Persentase Missing Values : 
CustomerID     24.927802
Description     0.268311
Country         0.001476
UnitPrice       0.000923
InvoiceNo       0.000000
StockCode       0.000000
Quantity        0.000000
InvoiceDate     0.000000
dtype: float64


In [29]:
missing_data_countries = data[data.isnull().any(axis=1)]['Country'].value_counts()
print("Negara yang memiliki missing value : ")
print(missing_data_countries)

Negara yang memiliki missing value : 
United Kingdom    133600
EIRE                 711
Hong Kong            288
Unspecified          202
Switzerland          125
France                66
Israel                47
Portugal              39
Australia              5
Singapore              3
Canada                 3
Bahrain                2
Name: Country, dtype: int64


In [55]:
unique_customers_by_country = data.groupby('Country')['CustomerID'].unique()

for country, customer_ids in unique_customers_by_country.items():
    print(f"CustomerID unik di {country}:")
    print(customer_ids)
    print("-" * 50)

CustomerID unik di Australia:
[12431. 12386. 12434. 12415. 12393. 12388. 12422. 16321. 12424.]
--------------------------------------------------
CustomerID unik di Austria:
[12865. 12370. 12373. 12414. 12817. 12818. 12429. 12360. 12358. 12453.
 12374.]
--------------------------------------------------
CustomerID unik di Bahrain:
[   nan 12355. 12353.]
--------------------------------------------------
CustomerID unik di Belgium:
[12395. 12417. 12423. 12383. 12402. 12464. 12401. 12431. 12362. 12361.
 12866. 12407. 12399. 12408. 12447. 12463. 12394. 12876. 12449. 12380.
 12379. 12465. 12397. 12364. 12390.]
--------------------------------------------------
CustomerID unik di Brazil:
[12769.]
--------------------------------------------------
CustomerID unik di Canada:
[15388.    nan 17443. 17444. 17844.]
--------------------------------------------------
CustomerID unik di Channel Islands:
[14932. 14442. 14935. 14934. 14936. 14930. 14931. 14937. 14566.]
--------------------------------

In [56]:
for col in data.columns:
    if col == 'CustomerID':  
        continue
    
    if data[col].isnull().sum() > 0:  
        if data[col].dtype in ['int64', 'float64']: 
            median_value = data[col].median()
            data[col] = data[col].fillna(median_value)
            print(f"Kolom '{col}' (Numerik) diisi dengan median: {median_value:.2f}")
        elif data[col].dtype == 'object':  
            mode_value = data[col].mode()[0]
            data[col] = data[col].fillna(mode_value)
            print(f"Kolom '{col}' (Kategorikal) diisi dengan mode: '{mode_value}'")

data["CustomerID"] = data.groupby("Country")["CustomerID"].transform(
    lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 0)
)
        
data.isnull().sum()

Kolom 'Description' (Kategorikal) diisi dengan mode: 'WHITE HANGING HEART T-LIGHT HOLDER'
Kolom 'UnitPrice' (Numerik) diisi dengan median: 2.08
Kolom 'Country' (Kategorikal) diisi dengan mode: 'United Kingdom'


InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

Mengisi nilai NaN di kolom CustomerID dengan modus berdasarkan negara adalah pendekatan yang baik karena modus mewakili nilai yang paling sering muncul dalam suatu grup, sehingga pengisian ini tetap mempertahankan konsistensi dan relevansi data. Pendekatan berbasis negara memastikan bahwa data yang hilang diisi dengan nilai yang sesuai dengan pola grup tersebut, menghindari pemberian nilai sembarangan yang bisa mengganggu analisis. Jika semua nilai dalam grup adalah NaN, menggantinya dengan 0 adalah solusi praktis untuk menjaga kelengkapan dataset tanpa mempengaruhi hasil analisis atau pemodelan lebih lanjut.

In [57]:
unique_customers_by_country = data.groupby('Country')['CustomerID'].unique()

for country, customer_ids in unique_customers_by_country.items():
    print(f"CustomerID unik di {country}:")
    print(customer_ids)
    print("-" * 50)

CustomerID unik di Australia:
[12431. 12386. 12434. 12415. 12393. 12388. 12422. 16321. 12424.]
--------------------------------------------------
CustomerID unik di Austria:
[12865. 12370. 12373. 12414. 12817. 12818. 12429. 12360. 12358. 12453.
 12374.]
--------------------------------------------------
CustomerID unik di Bahrain:
[12355. 12353.]
--------------------------------------------------
CustomerID unik di Belgium:
[12395. 12417. 12423. 12383. 12402. 12464. 12401. 12431. 12362. 12361.
 12866. 12407. 12399. 12408. 12447. 12463. 12394. 12876. 12449. 12380.
 12379. 12465. 12397. 12364. 12390.]
--------------------------------------------------
CustomerID unik di Brazil:
[12769.]
--------------------------------------------------
CustomerID unik di Canada:
[15388. 17444. 17443. 17844.]
--------------------------------------------------
CustomerID unik di Channel Islands:
[14932. 14442. 14935. 14934. 14936. 14930. 14931. 14937. 14566.]
----------------------------------------------